In [2]:
import tensorflow as tf
import numpy as np
import os
import time
import cv2
import numpy as np
from tqdm import tqdm

In [3]:
112

112

In [5]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )

    input_name = graph.get_operations()[0].name+':0'
    output_name = graph.get_operations()[-1].name+':0'
    return graph, input_name, output_name

In [6]:
model_path = "/home/junior/Documents/seguidor_linha/modelo_9935.pb"
load_graph(model_path)

W1010 23:39:32.316612 140284579424064 deprecation.py:506] From <ipython-input-5-2055476cfb0e>:17: calling import_graph_def (from tensorflow.python.framework.importer) with op_dict is deprecated and will be removed in a future version.
Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.


(<tensorflow.python.framework.ops.Graph at 0x7f963cee0400>,
 'prefix/conv2d_1_input:0',
 'prefix/dense_3/Softmax:0')

In [7]:
resultado = []
def prepare_data_x(img_dir,label=False,invert_color=False):
    inicio = time.time()
    tipos = ['cruzamento', 'curva', 'parada', 'reta', 'zigzag']
    aux = []
#     resultado = []

#     print(img_dir)
    frame = cv2.imread(img_dir,0)
    img = frame
#     img = cv2.resize(frame, (150,150))
    if invert_color:
        img = (255-img)
    aux.append(img)
    img = np.array(aux)
    img = img/255
    img = img.reshape(img.shape[0], img.shape[1], img.shape[2], 1).astype('float32')
#     print("tempo antes pred:",time.time()-inicio)
#     resultado.append(img)
#     return np.array(resultado)
    return img
#     pred = model.predict(img)
#     print("tempo pos pred:",time.time()-inicio)
#     pred = np.around(pred, decimals = 2, out = None) 
#     print("tempo TOTAL:",time.time()-inicio)
#     return tipos[np.argmax(pred)],pred,frame

In [8]:
prepare_data_x("teste.jpg").shape

(1, 480, 640, 1)

In [9]:
tipos = ['cruzamento', 'curva', 'parada', 'reta', 'zigzag']
def predict(model_path,folder):
    # load tf graph
    tf_model,tf_input,tf_output = load_graph(model_path)
    # Create tensors for model input and output
    x = tf_model.get_tensor_by_name(tf_input)
    y = tf_model.get_tensor_by_name(tf_output) 
    num_outputs = y.shape.as_list()[0]
    
    fotos = (os.listdir(folder))
    predictions = np.zeros((len(fotos),len(tipos)))
    
    with tf.Session(graph=tf_model) as sess:
        for foto in tqdm(fotos):    
            tempo1 = time.time()
            y_out = sess.run(y, feed_dict={x: prepare_data_x(folder+foto)})
            print("tempo pred:",time.time()-tempo1)
            predictions = y_out[0]
            print(predictions)
            pred = np.around(predictions, decimals = 2, out = None) 
            print (pred)
            print(tipos[np.argmax(pred)])
        #     return tipos[np.argmax(pred)],pred,input_data
#     return predictions

In [24]:
predict(model_path,"/home/junior/Downloads/separado/curva/")

  4%|▍         | 10/250 [00:00<00:57,  4.20it/s]

tempo pred: 0.33472704887390137
[1.6718151e-12 1.0000000e+00 1.4789081e-11 3.1597602e-10 5.4868932e-11]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.01101231575012207
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010427236557006836
[2.8667737e-17 1.0000000e+00 5.1778379e-20 1.1129665e-15 3.4006287e-16]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.011155366897583008
[2.8635145e-26 1.0000000e+00 4.5141668e-31 8.8871387e-29 3.0972395e-17]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.009916067123413086
[1.3028483e-38 1.0000000e+00 0.0000000e+00 3.8930638e-36 2.2888010e-33]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010004043579101562
[4.3608696e-30 1.0000000e+00 6.8200107e-37 1.4632023e-36 2.1434356e-21]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010399818420410156
[5.4270778e-33 1.0000000e+00 2.2571391e-38 2.5904654e-26 9.8233990e-30]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.011013031005859375
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.009969711303710938
[0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+0

 12%|█▏        | 30/250 [00:00<00:26,  8.17it/s]

tempo pred: 0.01040339469909668
[1.2107825e-34 1.0000000e+00 0.0000000e+00 2.4547489e-32 4.5464978e-25]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.011389732360839844
[2.0337296e-26 1.0000000e+00 4.4461146e-31 4.9348216e-24 8.6212222e-25]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.009926557540893555
[2.8217488e-31 1.0000000e+00 0.0000000e+00 3.2691535e-32 4.1942136e-27]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010209798812866211
[7.3326518e-28 1.0000000e+00 5.1171402e-36 2.2414238e-27 4.7862420e-24]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.00960850715637207
[0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 2.7810891e-37]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010468721389770508
[0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 1.0732317e-31]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010045528411865234
[8.1856827e-11 1.0000000e+00 3.4030493e-12 3.2197683e-08 6.6114703e-10]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.00998544692993164
[0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 4.0161524e-36]
[0

 20%|██        | 50/250 [00:00<00:13, 15.29it/s]

...

tempo pred: 0.009637832641601562
[6.1499694e-17 1.0000000e+00 1.7403026e-13 3.0378884e-14 1.3413974e-09]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.011075258255004883
[3.1250730e-30 1.0000000e+00 7.2223115e-33 1.4180800e-23 3.1828941e-23]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.009696245193481445
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010104894638061523
[2.9585671e-38 1.0000000e+00 0.0000000e+00 0.0000000e+00 9.6358863e-32]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010121822357177734
[1.0422431e-28 1.0000000e+00 1.4556497e-33 9.8146256e-24 4.3893678e-26]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010099649429321289
[2.3220082e-33 1.0000000e+00 1.3084844e-26 2.6061944e-27 5.1431187e-25]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.00909733772277832
[0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 2.1045432e-38]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010220766067504883
[3.2139798e-17 1.0000000e+00 8.4524024e-19 1.5632656e-12 2.0040621e-13]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.009238243103027344

 96%|█████████▌| 239/250 [00:02<00:00, 92.65it/s]


[0. 1. 0. 0. 0.]
curva
tempo pred: 0.011379241943359375
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.009960651397705078
[0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00 7.776499e-36]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010644912719726562
[1.9384837e-17 1.0000000e+00 8.3284341e-21 2.3131883e-15 1.0453917e-15]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010121345520019531
[4.2770196e-09 8.8979053e-01 6.9476435e-08 1.1020907e-01 2.9395974e-07]
[0.   0.89 0.   0.11 0.  ]
curva
tempo pred: 0.009445905685424805
[1.0878553e-25 1.0000000e+00 0.0000000e+00 2.0235024e-38 1.8429630e-21]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010359764099121094
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.009415864944458008
[2.674763e-36 1.000000e+00 0.000000e+00 7.697887e-32 3.610295e-32]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010485649108886719
[2.1973863e-32 1.0000000e+00 3.0332629e-36 1.8550471e-27 5.7083999e-24]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.009095430374145508
[0.0000000e+00 1.0000000e+00 

100%|██████████| 250/250 [00:03<00:00, 82.85it/s]

tempo pred: 0.00999140739440918
[3.0682447e-29 1.0000000e+00 4.4202596e-15 1.7531520e-25 2.6867904e-20]
[0. 1. 0. 0. 0.]
curva
tempo pred: 0.010381698608398438
[1.17005775e-26 1.00000000e+00 2.87230164e-32 9.00986821e-26
 8.11490428e-26]
[0. 1. 0. 0. 0.]
curva


In [15]:
output_names = ['dense_3/Softmax']
input_names = ['conv2d_1_input']

In [ ]:
1

In [10]:
tipos = ['cruzamento', 'curva', 'parada', 'reta', 'zigzag']
def predict(model_path,folder):
    # load tf graph
    tf_model,tf_input,tf_output = load_graph(model_path)
    print(tf_input,tf_output)
    # Create tensors for model input and output
    x = tf_model.get_tensor_by_name(tf_input)
    y = tf_model.get_tensor_by_name(tf_output) 
    num_outputs = y.shape.as_list()[0]
    
    fotos = (os.listdir(folder))
    predictions = np.zeros((len(fotos),len(tipos)))
    
    with tf.Session(graph=tf_model) as sess:
        for foto in tqdm(fotos):    
            tempo1 = time.time()
            y_out = sess.run(y, feed_dict={x: prepare_data_x(folder+foto)})
            print("tempo pred:",time.time()-tempo1)
            predictions = y_out[0]
            pred = np.around(predictions, decimals = 2, out = None) 
            print(tipos[np.argmax(pred)])
        #     return tipos[np.argmax(pred)],pred,input_data
#     return predictions

In [12]:
predict("/home/junior/Documents/seguidor_linha/modelo_9935.pb","/home/junior/Downloads/separado/parada/")


  0%|          | 0/250 [00:00<?, ?it/s]

prefix/conv2d_1_input:0 prefix/dense_3/Softmax:0



  0%|          | 1/250 [00:00<03:08,  1.32it/s]

tempo pred: 0.7553539276123047
parada



  1%|          | 2/250 [00:01<02:42,  1.53it/s]

...

tempo pred: 0.4359314441680908
parada



  5%|▌         | 13/250 [00:06<01:47,  2.21it/s]

tempo pred: 0.4667844772338867
parada


  5%|▌         | 13/250 [00:06<01:59,  1.99it/s]


KeyboardInterrupt: 

In [17]:
pwd

'/home/junior/Documents/carro2'